<a href="https://colab.research.google.com/github/Twinkle-gawri/AnuvadSaar/blob/main/Anuvadsaar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade fsspec datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
from datasets import load_dataset
dataset = load_dataset("abisee/cnn_dailymail", "3.0.0")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
!pip install transformers

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn").to(device)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
def preprocess(batch):
    source = batch['article']
    target = batch['highlights']
    model_inputs = tokenizer(source, max_length=512, truncation=True,padding='max_length')

    with tokenizer.as_target_tokenizer():
      labels = tokenizer(target, max_length=128, truncation=True,padding='max_length')['input_ids']

    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in
    label_seq] for label_seq in labels]
    model_inputs["labels"] = labels
    return model_inputs


In [ ]:
encoded_dataset = dataset.map(preprocess, batched=True,remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# Define training arguments
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/bart-summarization",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    report_to="none"
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'], # Use encoded_dataset instead of df_source
    eval_dataset=encoded_dataset['test']   # Use encoded_dataset instead of df_source
)

In [ ]:
trainer.train()

Step,Training Loss
500,1.167200
1000,1.140200
1500,1.138200
2000,1.160600
2500,1.159900


In [ ]:
sample_input = dataset['test'][0]['article']

inputs = tokenizer(sample_input, return_tensors='pt', truncation=True,max_length=512).to(device)

summary_ids = model.generate(**inputs, max_length=128, min_length=30,length_penalty=2.0, num_beams=4)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
print("English Summary:", summary)

In [ ]:
!pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer


reference = dataset['test'][0]['article']
generated = summary

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'],use_stemmer=True)

# Compute scores
scores = scorer.score(reference, generated)
print(scores)

In [ ]:
# Translation
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

trans_tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indictrans2-en-indic-1B", trust_remote_code=True)
trans_model =AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-en-indic-1B", trust_remote_code=True)


In [ ]:
summary = summary
formatted_input = "eng_Latn hin_Deva " + summary

In [ ]:
inputs = trans_tokenizer(formatted_input, return_tensors="pt", padding=True,truncation=True, max_length=512)
outputs = trans_model.generate(**inputs, max_new_tokens=128)
translated_text = trans_tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Hindi Translation:", translated_text)